In [1]:
from functools import cache


import networkx as nx

In [ ]:
INPUT_TEST_ = """029A
980A
179A
456A
379A"""
INPUT_TEST = INPUT_TEST_.split('\n')
INPUT_TEST_N = [int(s.replace('A', '')) for s in INPUT_TEST]

with open('d21_in.txt', 'r') as f:
    INPUT_ = f.read()
INPUT = INPUT_.split('\n')
INPUT_N = [int(s.replace('A', '')) for s in INPUT]

In [3]:
GRID_N_ = """789
456
123
.0A"""
GRID_N = tuple(tuple(s) for s in GRID_N_.split('\n'))
GRID_D_ = """.^A
<v>"""
GRID_D = tuple(tuple(s) for s in GRID_D_.split('\n'))

DIRS = {"^": (-1, 0), ">": (0, 1), "v": (1, 0), "<": (0, -1)}

NUM_KEYS = "0123456789A"
DIR_KEYS = "<>^vA"

In [4]:
# The illustration at the following link is very
# helpful for the recursion in part 2
# https://www.reddit.com/r/adventofcode/comments/1hja685/2024_day_21_here_are_some_examples_and_hints_for/
#
# going bottom to top, each character mutates into a 
# longer string of characters at the upper level
#
# v<<A^>>AAv<A<A^>>AAvAA^<A>Av<A^>A<A>Av<A^>A<A>Av<<A>A^>AAvA^<A>A [human]
#    <   AA  v <   AA >>  ^ A  v  A ^ A  v  A ^ A   < v  AA >  ^ A [robot 3]
#        ^^        <<       A     >   A     >   A        vv      A [robot 2]
#                           4         5         6                A [keypad robot]


def in_bounds(v, map_):
    i, j = v
    return 0 <= i <= len(map_)-1 and 0 <= j <= len(map_[0])-1


def get_nbhs(v, map_):
    i, j = v
    nbhs = [(i+DIRS[d][0], j+DIRS[d][1]) for d in DIRS]
    return [
        n for n in nbhs
        if in_bounds(n, map_)
        and map_[n[0]][n[1]] != '.'
    ]


def get_dir(v, w):
    vi, vj = v
    wi, wj = w
    di, dj = wi-vi, wj-vj
    for d in DIRS:
        if DIRS[d] == (di, dj):
            return d
    return "?"


def get_vertex(vlabel, map_):
    for i in range(len(map_)):
        for j in range(len(map_[0])):
            if map_[i][j] == vlabel:
                return (i, j)
    return (1_000_000, 1_000_000)


@cache
def get_all_shortest_paths(ulabel, vlabel, G, type_='N'):
    if type_ == 'N':  # numbers pad
        u = get_vertex(ulabel, GRID_N)
        v = get_vertex(vlabel, GRID_N)
    elif type_ == 'D':  # directions pad
        u = get_vertex(ulabel, GRID_D)
        v = get_vertex(vlabel, GRID_D)
    else:
        raise ValueError("D or N")
    strings = ()
    for path in nx.all_shortest_paths(G, u, v):
        ds = ""
        for i in range(len(path)-1):
            s, t = path[i], path[i+1]
            ds += get_dir(s, t)
        strings += (ds+"A",)
    return strings


@cache
def get_all_shortest_paths_code(code, G, type_):
    strings = ("",)
    code = "A"+code
    for i in range(len(code)-1):
        strings_i = ()
        d1, d2 = code[i], code[i+1]
        s_tmp = get_all_shortest_paths(d1, d2, G, type_)
        for s1 in strings:
            for s2 in s_tmp:
                strings_i += (s1 + s2,)
        strings = strings_i
    return strings


# use nx.DiGraph as a shortcut 
G_num = nx.DiGraph()
for i in range(len(GRID_N)):
    for j in range(len(GRID_N[0])):
        u = (i, j)
        if GRID_N[i][j] != '.':
            for v in get_nbhs(u, GRID_N):
                G_num.add_edge(u, v)
G_dir = nx.DiGraph()
for i in range(len(GRID_D)):
    for j in range(len(GRID_D[0])):
        u = (i, j)
        if GRID_D[i][j] != '.':
            for v in get_nbhs(u, GRID_D):
                G_dir.add_edge(u, v)


DIR_PAIRS = {s1+s2: get_all_shortest_paths(s1, s2, G_dir, 'D')
             for s1 in DIR_KEYS
             for s2 in DIR_KEYS}
NUM_PAIRS = {s1+s2: get_all_shortest_paths(s1, s2, G_num, 'N')
             for s1 in NUM_KEYS
             for s2 in NUM_KEYS}


@cache
def len_after_mutation(s: str):
    len_ = 0
    s_ = "A"+s
    for i in range(len(s_)-1):
        st = s_[i:i+2]
        len_ += len(DIR_PAIRS[st][0])
    return len_


@cache
def mutate_code(code: str, type_: str = 'D') -> list[str]:
    if type_ == 'D':
        DICT = DIR_PAIRS
    else:
        DICT = NUM_PAIRS
    strings = ("",)
    code = "A"+code
    for i in range(len(code)-1):
        strings_i = ()
        st = code[i:i+2]
        s_tmp = DICT[st]
        for s1 in strings:
            for s2 in s_tmp:
                strings_i += (s1 + s2,)
        strings = strings_i
    return strings


@cache
def len_rec_code_d(code: str, level, max_level) -> list[str]:

    min_len = 0

    if level == max_level:
        code = "A"+code
        for i in range(len(code)-1):
            st = code[i:i+2]
            s_tmp = DIR_PAIRS[st]
            if len(s_tmp) == 1:
                min_len += len(s_tmp[0])
            else:
                # s_tmp has two strings in it
                min_len += min(len(s_tmp[0]), len(s_tmp[1]))
        return min_len

    code = "A"+code
    for i in range(len(code)-1):
        st = code[i:i+2]
        s_tmp = DIR_PAIRS[st]
        if len(s_tmp) == 1:
            min_len += len_rec_code_d(s_tmp[0], level+1, max_level)
        else:
            # s_tmp has two strings in it
            min_len += min(
                len_rec_code_d(s_tmp[0], level+1, max_level), 
                len_rec_code_d(s_tmp[1], level+1, max_level),
            )

    return min_len

In [5]:
print(NUM_PAIRS['A4'])
print(mutate_code('4', 'N'))
print('-----------------------------------------------')
print(NUM_PAIRS['A4'])
print(NUM_PAIRS['45'])
print(NUM_PAIRS['56'])
print(NUM_PAIRS['6A'])
print(mutate_code('456A', 'N'))
# print(get_all_shortest_paths_code('>A', G_dir, 'D'))

('^^<<A', '^<^<A', '<^^<A', '^<<^A', '<^<^A')
('^^<<A', '^<^<A', '<^^<A', '^<<^A', '<^<^A')
-----------------------------------------------
('^^<<A', '^<^<A', '<^^<A', '^<<^A', '<^<^A')
('>A',)
('>A',)
('vvA',)
('^^<<A>A>AvvA', '^<^<A>A>AvvA', '<^^<A>A>AvvA', '^<<^A>A>AvvA', '<^<^A>A>AvvA')


In [6]:
print(get_all_shortest_paths_code('^^<<A', G_dir, 'D'))
print(mutate_code('^^<<A', 'D'))

('<AAv<AA>^>A', '<AAv<AA>>^A')
('<AAv<AA>^>A', '<AAv<AA>>^A')


In [7]:
# should print
# <A^A>^^AvvvA, <A^A^>^AvvvA, and <A^A^^>AvvvA
# for code 029A
print(get_all_shortest_paths_code("029A", G_num, type_='N'))
print(mutate_code('029A', 'N'))

# should print 68
print(len_after_mutation('v<<A>>^A<A>AvA<^AA>A<vAAA>^A'))

('<A^A^^>AvvvA', '<A^A^>^AvvvA', '<A^A>^^AvvvA')
('<A^A^^>AvvvA', '<A^A^>^AvvvA', '<A^A>^^AvvvA')
68


In [8]:
print(mutate_code('379A', 'N'))

('^A^^<<A>>AvvvA', '^A^<^<A>>AvvvA', '^A<^^<A>>AvvvA', '^A^<<^A>>AvvvA', '^A<^<^A>>AvvvA', '^A<<^^A>>AvvvA')


In [9]:
min([
    len_rec_code_d(c, 1, 2) for c in mutate_code('379A', 'N')
])

64

Part 1

In [10]:
# in_ = INPUT_TEST
in_ = INPUT
total = 0
for code in in_:
    mins = set()
    shortest_paths_l1 = get_all_shortest_paths_code(
        code, G_num, 'N')
    for sp_l1 in shortest_paths_l1:
        shortest_paths_l2 = get_all_shortest_paths_code(
            sp_l1, G_dir, 'D')
        for sp_l2 in shortest_paths_l2:
            mins.add(
                len_after_mutation(sp_l2)
            )
    min_ = min(mins)
    total += int(code.replace('A', ''))*min_
total

278748

Part 2

In [11]:
in_ = INPUT
total = 0
for code in in_:
    min_ = min([
        len_rec_code_d(c, 1, 25) for c in mutate_code(code, 'N')
    ])
    total += int(code.replace('A', ''))*min_
total

337744744231414